In [1]:
import findspark
import os
import gc
#findspark.init() 
SPARK_HOME='/opt/cloudera/parcels/CDH/lib/spark'
findspark.init(SPARK_HOME)

import time
import math
import copy
import csv
import json
import os
import codecs
import subprocess
import numpy as np
from pyspark import SQLContext
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.functions import size, array_union,flatten,array_sort,coalesce,broadcast,collect_list, collect_set, udf, array_remove, log, lit, first, col, array, sort_array,split, explode, desc, asc, row_number,isnan, when, count
from pyspark.sql.types import *
import rtree
from pyspark.sql import Window
import geofeather
from pyspark.storagelevel import StorageLevel

from pyspark.sql import SparkSession
from pyspark import StorageLevel
import geopandas as gpd
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, FloatType, ArrayType, MapType
from shapely.geometry import Point
from shapely.geometry import Polygon

from sedona.register import SedonaRegistrator
from sedona.core.SpatialRDD import SpatialRDD, PointRDD, CircleRDD, PolygonRDD, LineStringRDD
from sedona.core.enums import FileDataSplitter
from sedona.utils.adapter import Adapter
from sedona.core.spatialOperator import KNNQuery
from sedona.core.spatialOperator import JoinQuery
from sedona.core.spatialOperator import JoinQueryRaw
from sedona.core.spatialOperator import RangeQuery
from sedona.core.spatialOperator import RangeQueryRaw
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.core.formatMapper import WkbReader
from sedona.core.formatMapper import WktReader
from sedona.core.formatMapper import GeoJsonReader
from sedona.sql.types import GeometryType
from sedona.core.enums import GridType
from sedona.core.SpatialRDD import RectangleRDD
from sedona.core.enums import IndexType
from sedona.core.geom.envelope import Envelope
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

os.environ['PYSPARK_PYTHON'] = "./environment/bin/python"
#os.environ['PYSPARK_PYTHON'] = "/home/qiany/.conda/envs/py37/bin/python"
os.environ['YARN_CONF_DIR'] = "/opt/cloudera/parcels/CDH/lib/spark/conf/yarn-conf"

In [2]:
tin_file = input("Here is a programe to extract boundary relations, please input the absolute or relative path to your .ts file:")

# get the directory, basename of the input file
print("\n********************")
tin_directory = os.path.dirname(tin_file)
print("tin_directory: ", tin_directory)

directory_type = input("Is the data stored in hdfs(0) or Tri_data(1) or Tetra_data (2):") or "2"

if directory_type == '0':
    directory = 'hdfs_data'
elif directory_type == '1':
    directory = 'Tri_data'
else:
    directory = 'Tetra_data'
    
tin_basename = os.path.basename(tin_file) # input_vertices_2.off
print("tin_basename: ", tin_basename)

tin_filename = os.path.splitext(tin_basename)[0] # input_vertices_2
print("tin_filename: ", tin_filename)

tin_extension = os.path.splitext(tin_basename)[1] # .off
print("tin_extension: ", tin_extension)

print("\n********************")
print("This is a TIN file in \"%s\" format" % tin_extension)

filtra = 'yes'

# allocate the number of executors, the number of cores per executor, and the amount of memory per executor
Num_executor = '64'
Num_core_per_executor = '5'
Memory_executor = '64g'
MemoryOverhead_executor = '8g'

# allocate the number of cores for the driver node
Num_core_per_driver = '5'
Memory_driver = '64g'
MemoryOverhead_driver = '32g'

# the default number of shuffle partitions
Num_shuffle_partitions = input("spark.sql.shuffle.partitions:")


********************
tin_directory:  /home/qiany/yuehui/pyspark/Tetra_mesh/data
tin_basename:  Lander_big.ts
tin_filename:  Lander_big
tin_extension:  .ts

********************
This is a TIN file in ".ts" format


In [3]:
'''
spark.executor.cores: # Number of concurrent tasks an executor can run, euqals to the number of cores to use on each executor
spark.executor.instances: # Number of executors for the spark application
spark.executor.memory: # Amount of memory to use for each executor that runs the task
spark.executor.memoryOverhead:
spark.driver.cores: # Number of cores to use for the driver process; the default number is 1
spark.driver.memory: # Amount of memory to use for the driver
spark.driver.maxResultSize: to define the maximum limit of the total size of the serialized result that a driver can store for each Spark collect action
spark.default.parallelism: # Default number of partitions in RDDs returned by transformations like join, reduceByKey, and parallelize when not set by user. It can be set as spark.executor.instances * spark.executor.cores * 2
spark.sql.shuffle.partitions: determine how many partitions are used when data is shuffled between nodes, e.g., joins or aggregations. usually 1~5 times of executor.instances * executor.cores
spark.memory.storageFraction: determines the fraction of the heap space that is allocated to caching RDDs and DataFrames in memory.
spark.kryoserializer.buffer.max: determine the maximum of data that can be serialized at once; this must be larger than any object we attempt to serialize
spark.rpc.message.maxSize: # Maximum message size (in MiB) to allow in "control plane" communication; generally only applies to map output size information sent between executors and the driver. To communicate between the nodes, Spark uses a protocol called RPC (Remote Procedure Call), which sends messages back and forth. The spark.rpc.message.maxSize parameter limits how big these messages can be. 
spark.sql.broadcastTimeout: Spark will wait for this amount of time before giving up on broadcasting a table. Broadcasting can take a long time if the table is large or if there is a shuffle operation before it.
spark.sql.autoBroadcastJoinThreshold: Spark will broadcast a table to all worker nodes when performing a join if its size is less than this value; -1 means disabling broadcasting
'''

date = time.strftime("%m,%d,%Y")
date_name = date.split(',')[0] + date.split(',')[1] + date.split(',')[2]

hour = time.strftime("%H,%M")
hour_name = hour.split(',')[0] + hour.split(',')[1]

spark_app_name = "TopoRela_CritPts_Tetra_Spark_" + tin_filename + '_' + date_name + '_' + hour_name
print("spark_app_name:", spark_app_name)

spark = SparkSession \
.builder \
.appName(spark_app_name) \
.master('yarn') \
.config("spark.serializer", KryoSerializer.getName) \
.config('spark.jars','sedona-core-2.4_2.11-1.0.0-incubating.jar,sedona-sql-2.4_2.11-1.0.0-incubating.jar,sedona-python-adapter-2.4_2.11-1.0.0-incubating.jar,sedona-viz-2.4_2.11-1.0.0-incubating.jar,geotools-wrapper-geotools-24.0.jar,graphframes-0.8.0-spark2.4-s_2.11.jar') \
.config('spark.executor.cores', Num_core_per_executor) \
.config('spark.executor.instances', Num_executor) \
.config('spark.executor.memory', Memory_executor) \
.config('spark.executor.memoryOverhead', MemoryOverhead_executor) \
.config('spark.driver.cores', Num_core_per_driver) \
.config('spark.driver.memory', Memory_driver) \
.config('spark.driver.memoryOverhead', MemoryOverhead_driver) \
.config('spark.driver.maxResultSize', '0') \
.config('spark.dynamicAllocation.enabled', 'false') \
.config('spark.network.timeout', '10000001s') \
.config('spark.executor.heartbeatInterval', '10000000s') \
.config('spark.sql.shuffle.partitions', Num_shuffle_partitions) \
.config("spark.default.parallelism", '400') \
.config("spark.kryoserializer.buffer.max", "1024mb") \
.config('spark.rpc.message.maxSize', '256') \
.config("spark.sql.broadcastTimeout", "36000") \
.config("spark.sql.autoBroadcastJoinThreshold", "-1") \
.config("spark.sql.objectHashAggregate.sortBased.fallbackThreshold", "-1") \
.config('spark.yarn.dist.archives', '/local/data/yuehui/py37.tar.gz#environment') \
.config("spark.python.profile", "true") \
.config("spark.eventLog.enabled", "true") \
.config("spark.eventLog.logStageExecutorMetrics", "true") \
.getOrCreate()

spark_app_name: TopoRela_CritPts_optimal_Lander_big_06192024_1230


### read input filesread input files

In [4]:
# read_tetra_order() is a function used to read tetrahedron from a csv file
def read_tetra_order(hdfs_tetra_origin):
    '''
    this function has two input parameters.
    filtra: 'yes' or 'no', yes means that the input csv file is ordered by default
    directory: a string denoting the directory to a tetrahedra file
    tin_filename: a string denoting the file name of a tetrahedra extension, e.g., 827_monviso
    '''
        
    schema_tetra_origin = StructType([ \
        StructField("tetra_order",IntegerType(),True), \
        StructField("r1",IntegerType(),True), \
        StructField("r2",IntegerType(),True), \
        StructField("r3",IntegerType(),True), \
        StructField("r4",IntegerType(),True), \
        StructField("r1_ele",FloatType(),True), \
        StructField("r2_ele",FloatType(),True), \
        StructField("r3_ele",FloatType(),True), \
        StructField("r4_ele",FloatType(),True) \
      ])

    df_tetra_origin = spark.read.format("csv") \
          .option("header", False) \
          .schema(schema_tetra_origin)\
          .load(hdfs_tetra_origin)
        
    return df_tetra_origin


# read tetrahedra
hdfs_tetra_origin = directory + "/" + tin_filename + '_filtra_tetra_sort.csv'

df_tetra_order = read_tetra_order(hdfs_tetra_origin)
df_tetra_order.printSchema()

Number of partitions for df_tetra_origin: 4959
root
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)
 |-- r4: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- r2_ele: float (nullable = true)
 |-- r3_ele: float (nullable = true)
 |-- r4_ele: float (nullable = true)



### globally obtain VT relation

In [5]:
# function to get VT directly from DF_T
def get_VT(df_tetra_order):
    df_tetra_order = df_tetra_order.withColumn("tetra", sort_array(F.array("r1", "r2", "r3", "r4"), False))
    df_VT_init_1 = df_tetra_order.select("r1","tetra")
    df_VT_init_2 = df_tetra_order.select("r2","tetra")
    df_VT_init_3 = df_tetra_order.select("r3","tetra")
    df_VT_init_4 = df_tetra_order.select("r4","tetra")
    
    df_VT_union12 = df_VT_init_1.union(df_VT_init_2)
    df_VT_union123 = df_VT_union12.union(df_VT_init_3)
    df_VT_union1234 = df_VT_union123.union(df_VT_init_4)
    
    df_VT = df_VT_union1234.groupBy('r1').agg(collect_list('tetra').alias('VT'))
    df_VT = df_VT.withColumnRenamed('r1', 'Ver')
    
    return df_VT

In [6]:
df_VT = get_VT(df_tetra_order)

df_VT.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



### locally obtain VF, VV, TE, relations

In [7]:
from itertools import combinations
def get_Crit_Rela_from_VT(Ver, VT):
    if VT:
        # store VF in multi_F, VV in pt_set
        multi_F = set()
        pt_set = set()
        TE = {}
        for tetra in VT:
            # VF
            # Generate combinations of 3 elements including Ver
            for comb in combinations(tetra, 3):
                if Ver in comb:
                    sorted_comb = sorted(comb, reverse=True)
                    multi_F.add(tuple(sorted_comb))
                    
            # VV
            pt_set.update(tetra)
            
            # TE
            TE[tuple(tetra)] = [[tetra[0], tetra[1]], [tetra[0], tetra[2]], [tetra[0], tetra[3]], [tetra[1], tetra[2]], [tetra[1], tetra[3]], [tetra[2], tetra[3]]]
            
        # VF
        VF = sorted(multi_F) # save more time when using list(multi_F)
        
        # VV
        pt_set.remove(Ver)
        VV = sorted(pt_set)
        
        return VF, VV, TE
    
# StructType for Forman relations
union_Crit_rela_schema = StructType([
    StructField("VF", ArrayType(ArrayType(IntegerType())),True), 
    StructField("VV", ArrayType(IntegerType()),True),
    StructField("TE", MapType(ArrayType(IntegerType()), ArrayType(ArrayType(IntegerType()))),True)
])

get_Crit_Rela_from_VT_udf = udf(get_Crit_Rela_from_VT, union_Crit_rela_schema)

df_Crit_rela_init = df_VT.withColumn("Crit_rela", get_Crit_Rela_from_VT_udf(df_VT.Ver, df_VT.VT))
df_Crit_rela = df_Crit_rela_init.select("Ver", "VT", col("Crit_rela.VF").alias("VF"), col("Crit_rela.VV").alias("VV"), col("Crit_rela.TE").alias("TE"))
# df_VF = df_VF.select("Ver", "VF")
df_Crit_rela.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VF: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VV: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- TE: map (nullable = true)
 |    |-- key: array
 |    |    |-- element: integer (containsNull = true)
 |    |-- value: array (valueContainsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)



In [8]:
def bool_boundary(VT, VF):
    # check if Ver is a boundary vertex or internal vertex
    if 3*len(VT) == 2*len(VF):
        # this is an internal vertex
        return False
    else:
        return True

bool_boundary_udf = udf(bool_boundary, BooleanType())

df_Crit_rela = df_Crit_rela.withColumn("BoundaryOrNot", bool_boundary_udf(df_Crit_rela.VT, df_Crit_rela.VF))
df_Crit_rela.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VF: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VV: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- TE: map (nullable = true)
 |    |-- key: array
 |    |    |-- element: integer (containsNull = true)
 |    |-- value: array (valueContainsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |-- BoundaryOrNot: boolean (nullable = true)



In [9]:
t0 = time.time()

num_row = df_Crit_rela.count()

t1 = time.time()
t_count_Crit_rela = t1 - t0
print("Time cost:", t_count_Crit_rela)
print("number of rows:", num_row)

Time cost: 573.2613158226013
number of rows: 1792989718


### extracting critical points

In [10]:
# define UnionFind class outside the UDF
# the UnionFind class is an implementation of Union-Find algorithm
class UnionFind:
    def __init__(self, size):
        self.root = list(range(size))
        self.rank = [1] * size

    def find(self, x):
        if self.root[x] != x:
            self.root[x] = self.find(self.root[x])
        return self.root[x]

    def union(self, x, y):
        rootX = self.find(x)
        rootY = self.find(y)
        if rootX != rootY:
            if self.rank[rootX] > self.rank[rootY]:
                self.root[rootY] = rootX
            elif self.rank[rootX] < self.rank[rootY]:
                self.root[rootX] = rootY
            else:
                self.root[rootY] = rootX
                self.rank[rootX] += 1

In [11]:
# define an UDF to extracting critical vertices
def compute_crit_pts(Ver, VV, TE, BoundaryOrNot):
    # store VV in two groups, one group for vertices whose scala value is greater than Ver, the other is lower than Ver
    above_threshold = []
    below_threshold = []
    for v in VV:
        if v > Ver:
            above_threshold.append(v)
        else: # v < Ver, v is unique so v will never be equal to Ver
            below_threshold.append(v)
    
    # filter TE to only store edges that are incident in Ver
    edges = []
    for key_tetra in TE:
        for e in TE[key_tetra]:
            if Ver in e:
                edges.append(e)
    
    def find_components(vertices):
        uf = UnionFind(len(vertices))
        vertex_to_index = {vertex: i for i, vertex in enumerate(vertices)}
        for v1, v2 in edges:
            if v1 in vertex_to_index and v2 in vertex_to_index:
                uf.union(vertex_to_index[v1], vertex_to_index[v2])
        component_map = {}
        for vertex in vertices:
            root = uf.find(vertex_to_index[vertex])
            if root not in component_map:
                component_map[root] = []
            component_map[root].append(vertex)
        return list(component_map.values())
    
    # upper_list is a list of lists, which stores all vertices in the link of Ver; those vertices have higher scalar values 
    upper_list = find_components(above_threshold)
    lower_list = find_components(below_threshold)
    
    if BoundaryOrNot == False:# this is an internal vertex
        # the current Ver is a maximum
        if len(upper_list) == 0:
            crit_type = 'maximum'
            return crit_type
            
        # the current Ver is a minimum
        elif len(lower_list) == 0:
            crit_type = 'minimum'
            return crit_type
        
        # the current Ver is a 1-saddle
        elif len(upper_list) == 1 and len(lower_list) == 1:
            crit_type = 'regular'
            return crit_type
        
        # the current Ver is a 1-saddle
        elif len(upper_list) == 2 and len(lower_list) == 1:
            crit_type = '1-saddle'
            return crit_type
        
        # the current Ver is a 2-saddle
        elif len(upper_list) == 1 and len(lower_list) == 2:
            crit_type = '2-saddle'
            return crit_type
        
        else:
            crit_type = 'monkey-saddle'
            return crit_type
        
    else: # this is a boundary vertex
        # the current Ver is a maximum
        if len(upper_list) == 0:
            crit_type = 'maximum'
            return crit_type
            
        # the current Ver is a minimum
        elif len(lower_list) == 0:
            crit_type = 'minimum'
            return crit_type
        
        # the current Ver is a 1-saddle
        elif len(upper_list) == 1 and len(lower_list) == 1:
            crit_type = 'regular'
            return crit_type
        
        # the current Ver is a 1-saddle
        elif len(upper_list) == 2 and len(lower_list) == 1:
            crit_type = '1-saddle'
            return crit_type
        
        # the current Ver is a 2-saddle
        elif len(upper_list) == 1 and len(lower_list) == 2:
            crit_type = '2-saddle'
            return crit_type
        
        else:
            crit_type = 'monkey-saddle'
            return crit_type

compute_crit_pts_udf = udf(compute_crit_pts, StringType())

df_Crit = df_Crit_rela.withColumn("Crit_Type", compute_crit_pts_udf(df_Crit_rela.Ver, df_Crit_rela.VV, df_Crit_rela.TE, df_Crit_rela.BoundaryOrNot))
df_Crit.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VF: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)
 |-- VV: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- TE: map (nullable = true)
 |    |-- key: array
 |    |    |-- element: integer (containsNull = true)
 |    |-- value: array (valueContainsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)
 |-- BoundaryOrNot: boolean (nullable = true)
 |-- Crit_Type: string (nullable = true)



In [12]:
t0 = time.time()

num_row = df_Crit.count()

t1 = time.time()
t_count_Crit = t1 - t0
print("Time cost:", t_count_Crit)
print("number of rows:", num_row)

Time cost: 541.9926292896271
number of rows: 1792989718


In [13]:
print("****************************")
print("Total time cost for preparing the relations:", t_count_Crit_rela)

print("****************************")
print("Time cost for classifying the critical points:", t_count_Crit)

print("****************************")
print("Total time cost:", t_count_Crit_rela + t_count_Crit)

****************************
Total time cost for preparing the relations: 573.2613158226013
****************************
Time cost for classifying the critical points: 541.9926292896271
****************************
Total time cost: 1115.2539451122284
